<a href="https://colab.research.google.com/github/GirishNaik711/CodeClauseInternship_Color_Detection_OpenCV/blob/main/color_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


For this basic project, we'll rely on the cv2 library to read and display our chosen image. We'll also implement a mouse click callback, allowing us to capture RGB values from clicked pixels. This information will be used to display the color's name and its RGB values. To streamline our process, we'll leverage the pandas library for efficient reading of a CSV file containing 865 color names and their associated RGB values

#The Following is the code for Detection on an Image using OpenCV


In [3]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [4]:
cols = ('color', 'color_name', 'hex', 'R','G','B')
df = pd.read_csv('/content/colors.csv', names = cols)
img = cv.imread('/content/colorpic.jpg')

In [5]:
df.head()

,color,color_name,hex,R,G,B
0,air_force_blue_raf,Air Force Blue (Raf),#5d8aa8,93,138,168
1,air_force_blue_usaf,Air Force Blue (Usaf),#00308f,0,48,143
2,air_superiority_blue,Air Superiority Blue,#72a0c1,114,160,193
3,alabama_crimson,Alabama Crimson,#a32638,163,38,56
4,alice_blue,Alice Blue,#f0f8ff,240,248,255


Within the RGB color space, colors are represented using 24 bits, resulting in a total of 16,581,375 unique colors. Given this vast array of colors, naming each one individually is impractical. In our CSV file, we've curated a list of 865 color names. However, due to this limitation, certain RGB values will correspond to the same name. Our task is to associate each RGB value with a name, which is accomplished by calculating the distance between colors and determining the nearest match.

In [6]:
def getColorName(R, G, B):
  minimum = 1000
  for i in range(len(df)):
    d = abs(R - int(df.loc[i, 'R'])) + abs(G - int(df.loc[i, 'G'])) + abs(B - int(df.loc[i,'B']))
    if (d < minimum):
      minimum = d
      color_name = df.loc[i, 'color_name']

  return color_name

Registering the Mouse Click

In [7]:
clicked = False
r = g = b = xpos = ypos = 0

In [8]:
def draw_function(event, x, y, flags, param):
    global r, g, b, xpos, ypos, clicked
    if event == cv.EVENT_LBUTTONDBLCLK:
        clicked = True
        xpos = x
        ypos = y
        b, g, r = img[y, x]
        b = int(b)
        g = int(g)
        r = int(r)

In [ ]:
cv.namedWindow('color_detection')
cv.setMouseCallback('color_detection', draw_function)

Executing Color Detection on the co-ordinates of the mouse click

In [ ]:
while(1):
  cv2_imshow(img)
  if (clicked):
    cv.rectangle(img, (20,20), (750, 60), (b,g,r) , -1)
    color_name = getColorName(r,g,b) + ' R='+str(r) + ' B='+str(b) + ' G='+str(g)
    cv.putText(img, color_name, (50,50), 2, 0.8, (255,255,255), 2, cv.LINE_AA)

    if(r + g + b >= 600):
      cv.putText(img, color_name, (50,50), 2, 0.8, (0,0,0), 2, cv.LINE_AA)

    clicked = False

    if cv.waitKey(20) & 0xFF == 27:
        break

cv.destroyAllWindows()



After examining the colors within the image, suppose we're interested in identifying all locations where a particular color appears.

To achieve this, we convert both the image and the specific color of interest into the HSV color space. We define a lower bound of [H-10, 100, 100] and an upper bound of [H+10, 255, 255]. This ensures that only colors falling within this range are retained when we employ the cv2.inRange() function

In [ ]:
hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

color = np.uint8([[[b,g,r]]])
hsv_color = cv.cvtColor(color, cv.COLOR_BGR2HSV)

lower_range = np.array([hsv_color[0][0][0] - 10, 100, 100])
upper_range = np.array([hsv_color[0][0][0] + 10, 255, 255])

mask = cv.inRange(hsv, lower_range, upper_range)

cv.imshow('mask', mask)
cv.waitKey(0)
cv.destroyAllWindows()